
## Delta Lake & Data Ingestion

- Getting data into databricks is comonly called data ingestion.
- data engineers or data warehouse managers are primarily responsible for data ingestion.

### Delta Lake

![delta lake](./images/delta-lake.png)

- the goal of data ingestion is to bring in files and data from external data sourceslike cloud storage and sql tables inro Delta Lake as Delta tabels.
- Delta Lake is an open-source protocol that databricks uses for the data layer.


### Delta Table


- Delta tables store data within a folder directory. Within that directory the data is stored as **Parquet files** .
- Delta adds delta logs that are stored as JSON files alogside the parquet files.
- delta logs keep track of all transactions on data and table versions.
- table states are maintained using the transaction logs. If data is inserted, deleted or updated in the table, Delta adds a transaction (log file) and the table stays updated and managed.

- The transaction log provides:

  - **ACID transactions** (atomicity, consistency, isolation, durability) for concurrent reads/writes.
  - **Table versioning** enabling **time travel** (querying historical data).
  

### Key Features of Delta Lake

* ACID transaction support for safe concurrent operations.
* DML operations (INSERT, UPDATE, DELETE, MERGE).
* Time travel to query or restore previous versions.
* Schema enforcement and evolution.
* Unified batch and streaming support.
* Optimizations and scalability.



In [0]:
USE CATALOG workspace;
USE SCHEMA `2235-wk3`;

In [0]:
SELECT current_catalog(), current_schema();


**Common Data Importing Methods for Data Analysts:**

- File Upload UI

- CTAS (Creat table as Select)

- COPY INTO 

- FROM read_files()



### Data Ingestion with CTAS and read_files() - BATCH Ingestion

- `CREATE TABLE AS (CTAS)` is used to create and populate tables using the results of a query.
- `read_files()` table-valued function enebles reading data of various file formats and provides additional options for data ingestion.

**Documentation:**

- [read_files](https://docs.databricks.com/aws/en/sql/language-manual/functions/read_files)


**Note:**

- a `_rescued_data` column is automatically added to capture any data that does not match the inferred schema.



In [0]:
%sh 

ls /Volumes/workspace/2235-wk3/orders


In [0]:
SELECT * FROM csv.`/Volumes/workspace/2235-wk3/orders`

In [0]:
SELECT * FROM 

read_files(

  '/Volumes/workspace/2235-wk3/orders',
  format => 'csv',
  inferSchema => 'true',
  header => 'true',
  escape => '"'
) LIMIT  10

In [0]:
CREATE TABLE orders_bronze
USING DELTA -- optional
SELECT * FROM 
read_files(

  '/Volumes/workspace/2235-wk3/orders',
  format => 'csv',
  inferSchema => 'true',
  header => 'true',
  escape => '"'
);

-- preview the table

SELECT * FROM orders_bronze;

In [0]:
DESCRIBE TABLE orders_bronze;

In [0]:
DESCRIBE TABLE EXTENDED orders_bronze;


#### tabels

- managed table - UC manages everything; even cloud storage.
  - discards metadata and deletes the associated data when table is dropped
  - format is delta
  - comes with new features, performance, simplicity, stricter access
- external table - external cloud location
  - discards meteadata only. does not dete the data
  - the path specified bt the `LOCATION` keyword
  - manually manages
  - format can be DELTA, CSV, JSON, AVRO, and PARQUET etc.,

In [0]:
%python

df = (spark.read.format("csv").load("/Volumes/workspace/2235-wk3/orders", header = True, inferSchema = True, escape = '"'))

(df.write.mode("overwrite").saveAsTable("orders_bronze_py"))

orders_bronze = spark.table("orders_bronze_py")

orders_bronze.display()



### Data Ingestion using COPY INTO - Incremental data ingestion

- `COPY INTO` allows to load data from a file location into Delta table. 
- re-triable and idempotent.
- new files in source location are added and files already loaded are skipped.

**Documentation**

[COPY INTO](https://docs.databricks.com/aws/en/sql/language-manual/delta-copy-into)


- `mergeSchema` copy option is used for schema evolution.



In [0]:
DROP TABLE IF EXISTS orders_bronze_ci;
CREATE TABLE orders_bronze_ci;

COPY INTO orders_bronze_ci
FROM '/Volumes/workspace/2235-wk3/orders'
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'escape' = '"')
COPY_OPTIONS ('mergeSchema' = 'true');


SELECT * FROM orders_bronze_ci;



In [0]:

COPY INTO orders_bronze_ci
FROM '/Volumes/workspace/2235-wk3/orders'
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'escape' = '"')
COPY_OPTIONS ('mergeSchema' = 'true');


-- SELECT * FROM orders_bronze_ci;